In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import scipy
import matplotlib.pyplot as plt
from pathlib import Path
from read_data import get_network_df, get_corrs, get_full_df
from numba.core import types
import graph_tool
import numba
from numba import njit
from graph_tool.topology import shortest_distance
import pandas as pd
import networkx as nx
import graph_tool as gt
from graph_tool.topology import shortest_distance
import numpy as np
import typing

import numba
from numba.core import types

from transport_problem import OptimParams, DualOracle, HyperParams
from subgd import OracleStacker, ustm_mincost_mcf

In [2]:
T_LEN = 76
LA_LEN = 25
MU_LEN = 25

In [3]:
net_df = get_network_df(Path('SiouxFalls') / 'SiouxFalls_net.tntp')
corrs = get_corrs(Path('SiouxFalls') / 'SiouxFalls_trips.tntp') / 2

graph = graph_tool.Graph(net_df.values, eprops=[('capacity', 'double'), ('fft', 'double')])

In [4]:
# from graph_tool.draw import graph_draw
# graph_draw(graph)

In [5]:
source = 0
targets = np.arange(24)
weights = graph.ep.fft

short_distances, pred_map = shortest_distance(graph, source=source, target=targets, weights=weights,
                                              pred_map=True)

In [6]:
pred_map.a

PropertyArray([ 0,  0,  0,  2,  3,  1,  7,  5,  4,  8, 11,  2, 11, 10, 13,
                7, 15,  6, 16, 17, 23, 20, 23, 12])

In [7]:
from transport_problem import HyperParams, DualOracle, OptimParams 
l = np.sum(corrs, axis=1)
w = np.sum(corrs, axis=0)

zones_num = len(l)
sources = np.arange(zones_num)
targets = np.arange(zones_num)

params = HyperParams(gamma=10, mu_pow=0.25, rho=0.15)
oracle = DualOracle(graph, net_df, l, w, params)

In [8]:
oracle_stacker = OracleStacker(oracle, graph, sources, targets)

In [9]:
t, flows_averaged, dgap_log, cons_log, A_log, hist = ustm_mincost_mcf(oracle_stacker, 1e-2, None, 1000, None)

1
1
1
start optimizing


  0%|          | 0/1000 [00:00<?, ?it/s]/home/jupyter/transport/transport_network_optimization/transport_problem.py:83: RuntimeWarning: overflow encountered in exp
  return np.sum(np.exp(-T + optim_params.la[..., None] + optim_params.mu[None, ...]), axis=1) / np.sum(
/home/jupyter/transport/transport_network_optimization/transport_problem.py:84: RuntimeWarning: overflow encountered in exp
  np.exp(-T + optim_params.la[..., None] + optim_params.mu[None, ...])) - self.l
/home/jupyter/transport/transport_network_optimization/transport_problem.py:83: RuntimeWarning: invalid value encountered in divide
  return np.sum(np.exp(-T + optim_params.la[..., None] + optim_params.mu[None, ...]), axis=1) / np.sum(
/home/jupyter/transport/transport_network_optimization/transport_problem.py:87: RuntimeWarning: overflow encountered in exp
  return np.sum(np.exp(-T + optim_params.la[..., None] + optim_params.mu[None, ...]), axis=0) / np.sum(
/home/jupyter/transport/transport_network_optimization/transpor

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist)

In [ ]:
graph = None # TODO создавать граф
oracle = None # TODO - создать оракла
sources, targets = None, None    # определять sources и targets
oracle_stacker = OracleStacker(oracle, graph, sources, targets)
